In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

23/07/17 10:57:56 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.242.128 instead (on interface ens33)
23/07/17 10:57:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/17 10:57:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/17 10:57:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.json("../../curso2/datalake/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{87, 43, 2023-07...|{[{2023-07-14T17:...|{1234567890abcdef}|  2023-07-14|
|[{30, 83, 2023-07...|{[{2023-07-14T11:...|{1234567890abcdef}|  2023-07-14|
|[{0, 12, 2023-07-...|{[{2023-07-14T06:...|              null|  2023-07-14|
|[{87, 99, 2023-07...|{[{2023-07-16T17:...|{1234567890abcdef}|  2023-07-16|
|[{16, 30, 2023-07...|{[{2023-07-16T17:...|              null|  2023-07-16|
|[{76, 51, 2023-07...|{[{2023-07-13T06:...|{1234567890abcdef}|  2023-07-13|
|[{32, 55, 2023-07...|{[{2023-07-13T15:...|              null|  2023-07-13|
|[{90, 5, 2023-07-...|{[{2023-07-15T20:...|              null|  2023-07-15|
|[{16, 65, 2023-07...|{[{2023-07-12T14:...|              null|  2023-07-12|
|[{57, 89, 2023-07...|{[{2023-07-11T06:...|              null|  2023-07-11|
+-----------

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
from pyspark.sql import functions as f

In [15]:
df.select(f.explode("includes.users")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [16]:
df.select(f.explode("includes.users")).show(100)

+--------------------+
|                 col|
+--------------------+
|{2023-07-14T17:08...|
|{2023-07-14T02:24...|
|{2023-07-14T11:38...|
|{2023-07-14T15:16...|
|{2023-07-14T08:40...|
|{2023-07-14T00:39...|
|{2023-07-14T13:38...|
|{2023-07-14T19:45...|
|{2023-07-14T04:42...|
|{2023-07-14T10:43...|
|{2023-07-14T11:36...|
|{2023-07-14T04:26...|
|{2023-07-14T18:00...|
|{2023-07-14T20:46...|
|{2023-07-14T10:40...|
|{2023-07-14T22:58...|
|{2023-07-14T04:02...|
|{2023-07-14T14:08...|
|{2023-07-14T16:16...|
|{2023-07-14T16:24...|
|{2023-07-14T06:03...|
|{2023-07-14T19:13...|
|{2023-07-14T23:27...|
|{2023-07-14T23:04...|
|{2023-07-14T08:29...|
|{2023-07-14T12:25...|
|{2023-07-14T13:22...|
|{2023-07-14T21:52...|
|{2023-07-14T06:04...|
|{2023-07-14T18:50...|
|{2023-07-16T17:57...|
|{2023-07-16T04:41...|
|{2023-07-16T07:48...|
|{2023-07-16T05:25...|
|{2023-07-16T08:38...|
|{2023-07-16T08:55...|
|{2023-07-16T02:18...|
|{2023-07-16T05:44...|
|{2023-07-16T16:39...|
|{2023-07-16T04:46...|
|{2023-07-1

In [18]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [19]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [20]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-07-14T17:08:...| 11|User 1|   user1|
|2023-07-14T02:24:...| 52|User 2|   user2|
|2023-07-14T11:38:...| 93|User 3|   user3|
|2023-07-14T15:16:...|  3|User 4|   user4|
|2023-07-14T08:40:...| 24|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [21]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')